In [ ]:
import os
import pandas as pd
import numpy as np
import functools
import math
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.preprocessing import RobustScaler, OneHotEncoder
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split, learning_curve, RandomizedSearchCV
from sklearn.feature_selection import RFECV
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC, SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import RocCurveDisplay, roc_auc_score, f1_score, classification_report

In [ ]:
import warnings
warnings.simplefilter("ignore", UserWarning)

In [ ]:
data = {dataset_name.replace('.xlsx', ''): pd.read_excel(f'Data/{dataset_name}') for dataset_name in os.listdir('Data') if 'xlsx' in dataset_name} 

print(list(data.keys()))

In [ ]:
data_frames = [data['train_demo'], data['train_habits'], data['train_health']]
df = functools.reduce(lambda  left,right: pd.merge(left,right,on=['PatientID'], how='outer'), data_frames).set_index('PatientID')

data_frames = [data['test_demo'], data['test_habits'], data['test_health']]
df_test = functools.reduce(lambda  left,right: pd.merge(left,right,on=['PatientID'], how='outer'), data_frames).set_index('PatientID')

---
# Baseline

- only numeric features

In [ ]:
x, y = df.drop(columns = ['Disease']), df['Disease']

xnumeric = x.select_dtypes(include = np.number)

xtrain, xval, ytrain, yval = train_test_split(xnumeric, y, random_state = 111 ,test_size = 0.3, shuffle = True , stratify = y)

xtrain_index = xtrain.index
xval_index = xval.index

model = LogisticRegression(max_iter = 500, random_state = 1)
model.fit(xtrain,ytrain)

ypred = model.predict(xval)

f1_score(yval, ypred)

---
# Preprocessing

In [ ]:
def cleaning(dataframe):

    # fix Birth_Year errors (1869 > 1969)
    dataframe['Birth_Year'] = [i + 100 if i < 1900 else i for i in dataframe['Birth_Year']]

    # add Age column
    dataframe['Age'] = [2022 - i for i in dataframe['Birth_Year']]

    # add Gender column (1: Male, 0: Female)
    # afterwards drop column Name
    dataframe['Gender']  = [1 if i.split(' ')[0] == 'Mr.' else 0 for i in dataframe['Name']]
    dataframe.drop(columns = ['Name'], inplace = True)

    # add column population density
    dataframe['Region'] = [i.lower() for i in dataframe['Region']]
    dataframe['Region_Density'] = [dataframe['Region'].value_counts(normalize = True)[i] for i in dataframe['Region']]

    # encode Smoking_Habit & Exercise to binary (1: Yes, 0: No)
    dataframe['Smoking_Habit'] = [1 if i == 'Yes' else 0 for i in dataframe['Smoking_Habit']]
    dataframe['Exercise'] = [1 if i == 'Yes' else 0 for i in dataframe['Exercise']]

    # impute missing values in column "Education" with mode
    dataframe['Education'].fillna(dataframe['Education'].mode()[0], inplace=True)

    # remove outliers with IQR method
    dfnumeric = dataframe.select_dtypes(include = np.number)
    for i in dfnumeric.columns:
        Q1, Q3, IQR = dataframe[i].quantile(0.25), dataframe[i].quantile(0.75), dataframe[i].quantile(0.75) - dataframe[i].quantile(0.25)
        lower_whisker, upper_whisker = Q1 -(1.5*IQR), Q3 + (1.5*IQR)
        dataframe[i] = np.where(dataframe[i] > upper_whisker, upper_whisker, np.where(dataframe[i] < lower_whisker, lower_whisker, dataframe[i]))

    return dataframe

In [ ]:
def preprocessing(dataframe):
    # split x in numeric and categorical features
    xnumeric, xcategorical = dataframe.select_dtypes(include = np.number), dataframe.select_dtypes(exclude = np.number)

    # One-hot-encoding categorical features
    encoder = OneHotEncoder(handle_unknown='ignore')
    x_cat_encoded = pd.DataFrame(encoder.fit_transform(xcategorical).toarray(), columns = encoder.get_feature_names_out(), index = xcategorical.index) 

    # Scale numeric features
    scaler = RobustScaler()
    x_num_scaled = pd.DataFrame(scaler.fit_transform(xnumeric), index = xnumeric.index ,columns = xnumeric.columns)

    #PCA
    pca = PCA()
    pca_feat = pca.fit_transform(x_num_scaled)
    nr_of_components = np.argmax(np.cumsum(pca.explained_variance_ratio_) > 0.8) + 1

    pca = PCA(n_components=nr_of_components)
    pca_feat = pca.fit_transform(x_num_scaled)
    pca_feat_names = [f"PC{i}" for i in range(pca.n_components_)]
    pca_df = pd.DataFrame(pca_feat, index=x_num_scaled.index, columns=pca_feat_names)

    x_num_scaled_pca = pd.concat([x_num_scaled, pca_df], axis=1)

    xpreprocessed = pd.concat([x_num_scaled_pca, x_cat_encoded], axis = 1)

    return xpreprocessed

In [ ]:
x = preprocessing(cleaning(x))
y = y.loc[x.index]

xtrain, xval, ytrain, yval = x.loc[xtrain_index], x.loc[xval_index], y.loc[xtrain_index], y.loc[xval_index]

model = LogisticRegression(max_iter = 500, random_state = 1)
model.fit(xtrain,ytrain)

ypred = model.predict(xval)

f1_score(yval, ypred)

---
# Try better model

In [ ]:
model = RandomForestClassifier(random_state = 2)
model.fit(xtrain,ytrain)

ypred = model.predict(xval)

print(f'F1 score with RandomForest: {f1_score(yval, ypred)}')

---
# Try embedded feature selection

In [ ]:
from sklearn.feature_selection import SelectFromModel

In [ ]:
forest = RandomForestClassifier(random_state = 2)
forest.fit(xtrain, ytrain)
sfm = SelectFromModel(forest, prefit=True)

In [ ]:
xtrain_fs = sfm.transform(xtrain)
xval_fs = sfm.transform(xval)

model = RandomForestClassifier(random_state = 2)
model.fit(xtrain_fs, ytrain)

ypred = model.predict(xval_fs)

print(f'F1 score with SelectFromModel: {f1_score(yval, ypred)}')

# Try RFE

In [ ]:
model = RandomForestClassifier(random_state = 2)
rfe = RFECV(model, cv = 5, scoring = 'f1')
rfe.fit(xtrain, ytrain)
opt_features = list(rfe.get_feature_names_out(input_features = list(xtrain.columns)))

In [ ]:
xtrain_fs = rfe.transform(xtrain)
xval_fs = rfe.transform(xval)

model = RandomForestClassifier(random_state = 2)
model.fit(xtrain_fs, ytrain)

ypred = model.predict(xval_fs)

print(f'F1 score with RFE: {f1_score(yval, ypred)}')

---
# Hyperparameter tuning

In [ ]:
param_grid = {
                'bootstrap': [True, False],
                'max_depth': [50, 60, 70, 80, 90, 100, None],
                'max_features': ['auto', 'sqrt'],
                'n_estimators': [200, 400, 600, 800, 1000]
            }

model = RandomForestClassifier(random_state = 2)
search = RandomizedSearchCV(estimator = model, param_distributions = param_grid, scoring = 'f1', cv = 3, random_state = 3, n_jobs = -1)
search.fit(xtrain_fs, ytrain)

print(str(search.best_params_).replace('{','').replace('}','').replace("'","").replace(':','='))

In [ ]:
model = RandomForestClassifier(**search.best_params_, random_state = 2)
model.fit(xtrain_fs, ytrain)

ypred = model.predict(xval_fs)

print(f'F1 score with RandomizedSearch: {f1_score(yval, ypred)}')

# Try GridSearch instead of RandomizedSearch

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
forest = RandomForestClassifier(random_state = 2)
search = GridSearchCV(forest, param_grid, n_jobs = -1)
search.fit(xtrain_fs, ytrain)

In [ ]:
model = RandomForestClassifier(**search.best_params_, random_state = 2)
model.fit(xtrain_fs, ytrain)

ypred = model.predict(xval_fs)

print(f'F1 score with GridSearch: {f1_score(yval, ypred)}')

In [ ]:
from sklearn.metrics import confusion_matrix

confusion_matrix(yval, ypred)

In [ ]:
print(classification_report(yval, ypred))